In [1]:
import pandas as pd
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

In [2]:
read_final = True

if read_final:
    product = pd.read_pickle('./datathon/dataset/final_products')
    outfits = pd.read_pickle('./datathon/dataset/final_outfits')
else:
    o_outfits= pd.read_csv('./datathon/dataset/outfit_data.csv')
    o_product = pd.read_csv('./datathon/dataset/product_data.csv')

    outfits = o_outfits.copy()
    product = o_product.copy()

    #Deleting all the products that are not clothes
    product = product[(product.des_product_category!= 'Beauty') & (product.des_product_category!= 'Home')]
    # Delete all products for men and kids due to desbalance
    product = product[(product.des_line== 'SHE')]
    product = product[(product.des_sex== 'Female')]
    product = product[(product.des_age== 'Adult')]
    product = product[~product.des_product_aggregated_family.isin(['Dresses and jumpsuits','Swim and intimate','Accessories'])]
    product['des_product_category'] = product['des_product_category'].apply(lambda x: 'Tops' if x =='Outerwear' else x)
    product =product.drop(['des_sex','des_line','des_age'], axis = 1)

    # Delete the deleted products from outfits
    outfits = outfits[outfits.cod_modelo_color.isin(product.cod_modelo_color.unique())]

In [3]:
product.head()

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename,emb
0,41085800-02,02,OFFWHITE,WHITE,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2019_41085800_02.jpg,"[1.6967614, 1.6967614, 1.6967614, 1.6967614, 1..."
1,53000586-TO,TO,TEJANO OSCURO,BLUE,J-JEANS,Bottoms,Jeans,Jeans,Jeans,datathon/images/2019_53000586_TO.jpg,"[1.6821629, 1.6967614, 1.6967614, 1.6967614, 1..."
4,53070773-70,70,ROJO,RED,P-PLANA,Tops,Shirts,Shirt,Shirt,datathon/images/2019_53070773_70.jpg,"[1.6383677, 1.6383677, 1.6383677, 1.6383677, 1..."
5,53003736-94,94,GRIS MEDIO VIGORE,GREY,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53003736_94.jpg,"[1.5799739, 1.5799739, 1.5653756, 1.5653756, 1..."
6,53045718-17,17,CARAMELO,YELLOW,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53045718_17.jpg,"[1.5507771, 1.5653756, 1.5799739, 1.5799739, 1..."


## AUX FUNCTIONS 

In [4]:
def getProductIdx(idx):
    print("DATOS DEL PRODUCTO")
    print(product.iloc[idx])
    print('OUTFITS APPEARING IN:')
    code = product.iloc[idx].cod_modelo_color
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product.loc[idx]['des_filename']
    img = cv2.imread(image_path)
    plt.imshow(img)


In [5]:
def getProductByCode(code):
    print("DATOS DEL PRODUCTO")
    print(product.loc[product.cod_modelo_color == code])
    print('OUTFITS APPEARING IN:')
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product[product.cod_modelo_color == code]['des_filename'].values[0]
    img = cv2.imread(image_path)
    plt.imshow(img)


In [6]:
def printOufit(outfit):
    listaPrednas =  outfits[outfits.cod_outfit==1]['cod_modelo_color'].unique()
    for i in listaPrednas:
        image_path =product[product.cod_modelo_color == i]['des_filename'].values[0]
        img = cv2.imread(image_path)
        plt.imshow(img)
        plt.show()

In [7]:
def getEmbeding(code):
    if code == None:
        return None
    return product[product.cod_modelo_color==code]['emb'].values

In [8]:
# Calulate probabilities
outfits['col'] = outfits['cod_modelo_color'].apply(lambda x: x.split('-')[1])
df_grouped = outfits.groupby('cod_outfit')['col'].apply(list).reset_index()

conteo_color = {}
for i in product["cod_color_code"].unique():
    conteo_color[i] = 0
    for j in df_grouped.iloc[:,1].values:
        for k in j:
            if k == i:
                for z in range(0,len(j)):
                    conteo_color[k+j[z]] = 0

for i in product["cod_color_code"].unique():
    conteo_color[i] = 0
    for j in df_grouped.iloc[:,1].values:
        for k in j:
            if k == i:
                conteo_color[i] += 1
                for z in range(0,len(j)):
                    if k != j[z]:
                        conteo_color[k+j[z]] += 1
                    else:
                        conteo_color[k+j[z]] += .5

In [9]:
# Probabilidad condiciional de code2 once you have code 1
def probabilidad_col(colorcode1, colorcode2):
    code = colorcode1 + colorcode2
    prob = conteo_color[code]/conteo_color[colorcode1]
    return prob

In [10]:
outfits['fab'] = outfits["cod_modelo_color"].apply(lambda x: product[(product['cod_modelo_color'] == x)].iloc[0][7])
df_grouped_fab = outfits.groupby('cod_outfit')['fab'].apply(list).reset_index()

tejido = {}
for i in product["des_fabric"].unique():
    tejido[i] = 0
    for j in df_grouped_fab.iloc[:,1].values:
        for k in j:
            if k == i:
                for z in range(0,len(j)):
                    tejido[k+j[z]] = 0

for i in product["des_fabric"].unique():
    tejido[i] = 0
    for j in df_grouped_fab.iloc[:,1].values:
        for k in j:
            if k == i:
                tejido[i] += 1 * len(j)
                for z in range(0,len(j)):
                    if k != j[z]:
                        tejido[k+j[z]] += 1
                    else:
                        tejido[k+j[z]] += .5

C:\Users\ivans\AppData\Local\Temp\ipykernel_11096\1675487738.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  outfits['fab'] = outfits["cod_modelo_color"].apply(lambda x: product[(product['cod_modelo_color'] == x)].iloc[0][7])


In [11]:
def probabilidad_fab(fabric1, fabric2):
    fabric = fabric1 + fabric2
    try:
        prob = tejido[fabric]/tejido[fabric1]
        return prob
    except:
        return 0

# BUSQUEDA LOCAL

In [12]:
def computeHeuristicPorbs(codes):
    
    fab1 = product[product.cod_modelo_color==codes[0]]['des_fabric'].values[0]
    fab2 = product[product.cod_modelo_color==codes[1]]['des_fabric'].values[0]
    
    col1 = product[product.cod_modelo_color==codes[0]]['cod_color_code'].values[0]
    col2 = product[product.cod_modelo_color==codes[1]]['cod_color_code'].values[0]
    a=0
    b=0
    c=0
    d=0
    if len(codes) ==3:
        fab3 = product[product.cod_modelo_color==codes[2]]['des_fabric'].values[0]
        col3 = product[product.cod_modelo_color==codes[2]]['cod_color_code'].values[0]
        b=probabilidad_fab(fab1,fab3)
        d=probabilidad_col(col1,col3)
        
    a=probabilidad_fab(fab1,fab2)   
    c=probabilidad_col(col1,col2)
    
    mean= (a+b+c+d) /len(codes)
    return mean

In [13]:
computeHeuristicPorbs(['51000622-02','43067759-01'])

0.14753587024329382

In [14]:
def computeHeuristic(embedings):
    sum_distances=0
    for i in range(len(embedings)):
        for j in range(i + 1, len(embedings)):
            distance_ij = np.linalg.norm(embedings[i][0] - embedings[j][0])
            sum_distances += distance_ij
    return sum_distances

In [ ]:
def getNewProdSucc(estado, part):    
    data = product.copy()
    match part:
        case "top":
            data  = product.loc[product['des_product_category'] == "Tops"]
        case "bot":
            data =  product.loc[product['des_product_category'] == "Bottoms"]
        case "shoes":
            data  = product.loc[product['des_product_family'] == "Footwear"]
    
    randomObs = data.sample(1)
    newProd = randomObs.cod_modelo_color.values[0]
    return newProd

In [ ]:
MAX_DEPTH = 5

In [35]:
def hillClimbing(estado):
    successors = []
    random_seed = None
    
    # top sucesores
    topPart = None
    while (topPart == None):
        topPart = getNewProdSucc(estado, "top")
        newState = State(topPart, estado.get_bot(), estado.get_shoes(),depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH < estado.get_heuristica()):
            topPart = None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    
    # bottom sucesores
    bottomPart = None
    while(bottomPart == None):
        bottomPart = getNewProdSucc(estado, "bot")
        newState = State(estado.get_top(), bottomPart, estado.get_shoes(),depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH < estado.get_heuristica()):
            bottomPart = None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    # zapatos sucesores
    shoesPart = None
    while (shoesPart == None):
        shoesPart = getNewProdSucc(estado, "shoes")
        newState = State(estado.get_top(), estado.get_bot(), shoesPart, depth=(estado.get_depth()+1))
        
        newStateH = newState.get_heuristica()
        if(newStateH < estado.get_heuristica()):
            shoesPart = None
        else:
            successors.append((newState, newState.get_heuristica()))
        
    
    if (len(successors) == 0 or (estado.get_depth()) > 5): return estado
    
    # get first as best
    maxH = successors[0][1]
    bestSuccessor = successors[0][0]
    
    for succ in successors:
        if(maxH < succ[1]):
            maxH = succ[1]
            bestSuccessor = succ[0]
    print(bestSuccessor.get_depth())
    
    return hillClimbing(bestSuccessor)

In [36]:
class State:
    def __init__(self, top=None, bot=None, shoes=None, heuristica=np.inf, depth = 0):
        self.top = top
        self.bot = bot
        self.shoes = shoes
        self.heuristica = heuristica
        self._depth = depth
        if (self.top!= None or self.shoes!=None or self.bot!=None): self.heuristica = computeHeuristic(self.getAllEmbedings())
        print('DEPTH',self.get_depth())
        print('H',self.get_heuristica())
        
    def getAllEmbedings(self):
        embedings = [getEmbeding(self.top),getEmbeding(self.bot),getEmbeding(self.shoes)]
        embedings = [value for value in embedings if value is not None]
        return embedings

        
    def getProducts(self):
        fig, axs = plt.subplots(1, 3, figsize=(10, 3))  # Create a 1x3 grid of subplots

        # Top
        top_image_path = product[product.cod_modelo_color == self.top]['des_filename'].values[0]
        top_img = cv2.imread(top_image_path)
        axs[0].imshow(top_img)
        axs[0].set_title('Top')

        # Bottom
        bot_image_path = product[product.cod_modelo_color == self.bot]['des_filename'].values[0]
        bot_img = cv2.imread(bot_image_path)
        axs[1].imshow(bot_img)
        axs[1].set_title('Bottom')

        # Shoes
        shoes_image_path = product[product.cod_modelo_color == self.zapatos]['des_filename'].values[0]
        shoes_img = cv2.imread(shoes_image_path)
        axs[2].imshow(shoes_img)
        axs[2].set_title('Shoes')

        plt.show()
    def show_top(self):
        return getProductByCode(self.top)

    def show_bot(self):
        return getProductByCode(self.bot)

    def show_shoes(self):
        return getProductByCode(self.shoes)
    
    def get_top(self):
        return self.top

    def get_bot(self):
        return self.bot

    def get_shoes(self):
        return self.shoes
    
    def get_depth(self):
        return self._depth
    
    def get_heuristica(self):
        return self.heuristica

In [ ]:
outfit0= State(top='51000622-02',bot='43067759-01')

In [37]:
out_fin= hillClimbing(outfit0)

DEPTH 1
H 756.5162963867188
DEPTH 1
H 582.248046875
DEPTH 1
H 934.8560028076172
1
DEPTH 2
H 1411.6137390136719
DEPTH 2
H 1560.6476135253906
DEPTH 2
H 434.9778137207031
DEPTH 2
H 194.89169311523438
DEPTH 2
H 1158.035629272461
2
DEPTH 3
H 1672.249267578125
DEPTH 3
H 1937.2419738769531
DEPTH 3
H 1174.8620300292969
DEPTH 3
H 1257.2788009643555
DEPTH 3
H 1258.798583984375
DEPTH 3
H 1430.7543334960938
DEPTH 3
H 1580.0875854492188
3
DEPTH 4
H 1964.1965942382812
DEPTH 4
H 1728.0344543457031
DEPTH 4
H 1278.3713684082031
DEPTH 4
H 2001.5753479003906
DEPTH 4
H 1832.6581115722656
DEPTH 4
H 1637.8281555175781
DEPTH 4
H 1633.7733688354492
DEPTH 4
H 1633.3861999511719
DEPTH 4
H 1697.9786071777344
DEPTH 4
H 1633.7733688354492
DEPTH 4
H 1578.5182495117188
DEPTH 4
H 1634.876480102539
DEPTH 4
H 1690.1896057128906
DEPTH 4
H 1634.4797134399414
DEPTH 4
H 1619.6569366455078
DEPTH 4
H 1728.7235870361328
DEPTH 4
H 1731.4269256591797
DEPTH 4
H 1843.2417602539062
DEPTH 4
H 1630.128189086914
DEPTH 4
H 1761.298538

DEPTH 7
H 1727.8536529541016
DEPTH 7
H 1854.9065246582031
DEPTH 7
H 1964.4773254394531
DEPTH 7
H 1788.7843017578125
DEPTH 7
H 1861.3199768066406
DEPTH 7
H 1791.4090728759766
DEPTH 7
H 1908.4385070800781
DEPTH 7
H 1763.874382019043
DEPTH 7
H 1836.3132781982422
DEPTH 7
H 1821.4569396972656
DEPTH 7
H 1839.8706665039062
DEPTH 7
H 1746.603172302246
DEPTH 7
H 1800.2867279052734
DEPTH 7
H 2014.9254150390625
DEPTH 7
H 1762.5910415649414
DEPTH 7
H 1843.0773620605469
DEPTH 7
H 1862.6192932128906
DEPTH 7
H 1784.8296508789062
DEPTH 7
H 2006.0873107910156
DEPTH 7
H 1832.1369934082031
DEPTH 7
H 1807.511215209961
DEPTH 7
H 1835.9277038574219
DEPTH 7
H 1734.7465362548828
DEPTH 7
H 1893.9417114257812
DEPTH 7
H 2018.9791259765625
DEPTH 7
H 1782.1727752685547
DEPTH 7
H 1914.8133544921875
DEPTH 7
H 1853.5016326904297
DEPTH 7
H 1803.6021423339844
DEPTH 7
H 1934.5502624511719
DEPTH 7
H 2026.53759765625
DEPTH 7
H 1778.991096496582
DEPTH 7
H 1829.5535888671875
DEPTH 7
H 1886.48828125
DEPTH 7
H 1764.1675720214

In [43]:
top = out_fin.get_top()
bot = out_fin.get_bot()
shoes = out_fin.get_shoes()

In [46]:
product

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename,emb
0,41085800-02,02,OFFWHITE,WHITE,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2019_41085800_02.jpg,"[1.6967614, 1.6967614, 1.6967614, 1.6967614, 1..."
1,53000586-TO,TO,TEJANO OSCURO,BLUE,J-JEANS,Bottoms,Jeans,Jeans,Jeans,datathon/images/2019_53000586_TO.jpg,"[1.6821629, 1.6967614, 1.6967614, 1.6967614, 1..."
4,53070773-70,70,ROJO,RED,P-PLANA,Tops,Shirts,Shirt,Shirt,datathon/images/2019_53070773_70.jpg,"[1.6383677, 1.6383677, 1.6383677, 1.6383677, 1..."
5,53003736-94,94,GRIS MEDIO VIGORE,GREY,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53003736_94.jpg,"[1.5799739, 1.5799739, 1.5653756, 1.5653756, 1..."
6,53045718-17,17,CARAMELO,YELLOW,T-TRICOT,Tops,Sweaters and Cardigans,Sweater,Sweater,datathon/images/2019_53045718_17.jpg,"[1.5507771, 1.5653756, 1.5799739, 1.5799739, 1..."
...,...,...,...,...,...,...,...,...,...,...,...
9212,57091175-OR,OR,ORO,WHITE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Accessories,Jewellery,Necklace,datathon/images/2023_57091175_OR.jpg,"[1.8865409, 1.8865409, 1.8865409, 1.8865409, 1..."
9213,57085788-79,79,CALDERO,RED,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2023_57085788_79.jpg,"[1.8427455, 1.8281472, 1.8281472, 1.8427455, 1..."
9216,57094435-30,30,MARRON,BROWN,T-TRICOT,Tops,Sweaters and Cardigans,Cardigans,Cardigan,datathon/images/2023_57094435_30.jpg,"[1.8427455, 1.8281472, 1.8281472, 1.8427455, 1..."
9218,57076028-92,92,GRIS,GREY,P-PLANA,Bottoms,Skirts and shorts,Shorts,Shorts,datathon/images/2023_57076028_92.jpg,"[1.6675645, 1.6675645, 1.6675645, 1.6675645, 1..."


In [47]:
product[product.cod_modelo_color==top]

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename,emb
3786,51000622-02,02,OFFWHITE,WHITE,K-CIRCULAR,Tops,Tops,Tops,Top,datathon/images/2019_51000622_02.jpg,"[1.6967614, 1.6967614, 1.6967614, 1.6967614, 1..."
